The ESC403 cluster comes with [Catherine Devlin's `%sql`-magic for IPython][1]; this allows you to run SQL queries from the IPython notebook, and intermix them with Python code.

[1]: https://github.com/catherinedevlin/ipython-sql

Before we can use the `%sql` syntax, two steps must be taken:

* Load the IPython-SQL bridge code

In [1]:
%load_ext sql


* Connect to an actual database; this must be the first `%sql` statement (the funny `rmurri@/lustre` syntax is correct and means "connect to the PostGreSQL DB named `lustre` running on *this* host as user `rmurri`"  -- please replace `rmurri` with your local user name):

In [14]:
%sql postgresql://mivkov@/lustre


u'Connected: mivkov@lustre'

Now you can run 1-line SQL queries by prefixing them with `%sql`:

In [3]:
%sql select * from lustre limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path
usr388,i5105,1384455829,1384455829,4,1653,/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose/LIBO00323201_dock009.pdb
usr388,i5105,1384453069,1384453069,4,1378,/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose/LIBO00195993_dock004.pdb
usr388,i5105,1384454883,1384454883,4,2038,/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose/LIBO00271731_dock012.pdb
usr388,i5105,1384450216,1384450216,4,1873,/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose/LIBO00107714_dock008.pdb
usr388,i5105,1384457084,1384457084,4,1873,/scratch/bioc/usr388/VS_AllNow_libo_3WAT/ledock_pose/LIBO00369091_dock017.pdb


It is also possible to run multi-line (or multiple) SQL queries by using the `%%sql` syntax instead.  Note that in this case the SQL instructions *must not* be on the same line as the `%%sql` magic marker:

In [6]:
%%sql
select count(*) from lustre;
select distinct count(usr) from lustre;
select distinct count(grp) from lustre;

1 rows affected.
1 rows affected.
1 rows affected.


count
30356776


The `%%time` magic prints the time taken to evaluate a cell (which comes handy when doing performance comparisons):

In [8]:
%%time

import time
time.sleep(5)

CPU times: user 0 ns, sys: 3.76 ms, total: 3.76 ms
Wall time: 5.01 s


----

**Note:** to keep running times low, we will be using table `lustre_sample` throughout, which contains a sample of 5% the rows of the original `lustre` table.

## 1. How many files are there in the `lustre` dataset

In [20]:
%%time
%%sql
select count(distinct path)
from lustre_sample



1 rows affected.
CPU times: user 4.39 ms, sys: 1.54 ms, total: 5.93 ms
Wall time: 1min 13s


count
1519053


Since we know that all values in the `path` column are distinct, we can omit the `DISTINCT` keyword. Does it gain us some performance?

In [21]:
%%time
%%sql
select count(path)
from lustre_sample

1 rows affected.
CPU times: user 1.74 ms, sys: 0 ns, total: 1.74 ms
Wall time: 236 ms


count
1519053


It saved us massive amounts of time.

## 2. How many distinct users own files in the dataset?

In [22]:
%%sql
select count(distinct usr)
from lustre_sample

1 rows affected.


count
40


## 3. What user has the largest number of files?

In [94]:
%%sql
select usr, count(path) as "number of files"
from lustre_sample
    group by usr
    order by count(path) desc
limit 1

1 rows affected.


usr,number of files
usr356,602120


## 4. Which is/are the most-recently accessed file(s)?

The `atime` field stores the last-access time in UNIX "epoch" format.
The *maximum* `atime` value is the most recent one::

In [99]:
%%sql
select atime, path
from lustre_sample
order by atime desc
limit 1

1 rows affected.


atime,path
1393664299,/scratch/bioc/usr384/TD-RS/run08/equi/output/restart.xsc


Then you should of course select the path(s) with that access time::

Other option, in two steps:

In [100]:
%sql select max(atime) from lustre_sample

1 rows affected.


max
1393664299


If we already know the *number* of files with maximum `atime`, we can
use a single query to get their names::

In [101]:
%sql select path from lustre_sample where atime=1393664299

1 rows affected.


path
/scratch/bioc/usr384/TD-RS/run08/equi/output/restart.xsc


Just in case you wonder: `atime=max(atime)` is not a valid `WHERE` clause::

## 5. Let's say that $T_0$ is the time the most-recently accessed file was last read. How many files were not read in the month preceding $T_0$?

We know that T0=1393671122. Now it's easy to select paths accessed in
the given range::

In [105]:
%sql select count(path) from lustre_sample where atime < (1393671122-(24*30*3600))

1 rows affected.


count
1493228


## 6. Is it possible to convert fields atime and mtime to PostgreSQL's TIMESTAMP type?

Yes, it *is* possible to alter a SQL table definition after the table has been created.

We shall now show how to (1) create a new table, (2) populate it, then (3) alter the definition and (4) fill the new column with values.

In [ ]:
# (1)+(2) create a new table and populate it


In [ ]:
# (3) alter table definition


In [ ]:
#(4) populate additional column


Show some data from the table we created::

## 7. Can you count the number of files in a given directory?

Yes, using SQL's `like` string matching operator, which allows any
part of a string to be matched by the `%` character (i.e., just like
`*` for file names)::

## 8. Can you find the directory that holds the largest number of files? 

## 9. Can you find the directory tree that holds the largest number of files?

Yes or no? *(and why?)*